This notebook walks the user through the process of creating a sample dataset of clustered data that features a response variable - Y, a dichotomous treatment variable - T, random noise in the form of epsilon, and 2 covariates that we will adjust for (X1 and X2) in the context of estimating: $$ Y \sim \beta_0 + \beta_1 T + \beta_2 X1 + \beta_3 X2 + \epsilon$$

In [7]:
import olspow as osp
from generate_data import gen_clustered_data

In [2]:
df = gen_clustered_data(n_c=100000,
                        mu_nobs_per_c=1, std_nobs_per_c=2,
                        mu_of_mu_c_treated=4, std_of_mu_c_treated=0.5,
                        mu_of_mu_c_cov_1=100, std_of_mu_c_cov_1=10,
                        mu_of_mu_c_cov_2=10, std_of_mu_c_cov_2=1,
                        randomization_seed=1)

In the data generating process called above, the arguments *n_c*, *mu_nobs_per_c*, and *std_nobs_per_c* will produce 100,000 clusters of data with a mean number of 1 observation / row per cluster and a standard deviation of 2 observations per cluster. The arguments *mu_of_mu_c_treated* and *std_of_mu_c_treated* mean that our clusters will experience heterogeneous treatment effects where mean treatment effect is a 4 unit increase in response variable, *Y*, and that the standard deviation of this treatment effect is 0.5. The mean of our cluster means for our first covariate - X1 - will be 100, while the standard deviation of the cluster means is 10 (i.e. the *mu_of_mu_c_cov_1* and *std_of_mu_c_cov_1* arguments). Similiarly, the mean and standard deviation of the cluster means for the second covariate - X2 - will be 10 and 1 respectively. We can see our resulting data below. 

In [3]:
df.head()

,ID,CLUSTER_ID,NOBS_IN_CLUSTER,Y,TREATED_CLUSTER,TREATMENT_EFFECT,X1_VALUE,X1_COEF,X2_VALUE,X2_COEF,EPSILON
0,0,0,1,514.346106,1,5.080007,98.622225,4.994344,8.428242,1.649372,2.811456
1,1,1,3,608.451212,1,4.336182,113.221716,5.040928,10.558454,2.176741,10.389488
2,2,1,3,611.074277,1,3.947268,118.124514,5.039657,5.114475,2.563951,-1.293304
3,3,1,3,502.833341,1,4.078917,95.033426,5.042160,10.509967,2.482262,-6.507764
4,4,2,1,637.993807,0,0.000000,105.934036,5.784349,11.316836,2.027084,2.294180


Now we will call *olspow*'s primary method : *solve_power()*. The first argument - *is_ratio* - informs *olspow* that we are taking a ratio of sums (as opposed to a ratio between a sum and a count). We pass are simulated data to the *data* argument, and use the *endog*, *exog*, and *cluster* arguments to specify the response variable, covariates that we are adjusting for, the and the cluster key respectively. NOTE : *olspow* assumes that the cluster key and unit of experimental assignment are identical. Using the *ratio* argument, we specify that our test design will use balanced assignment across arms. *alpha* denotes our target false positive rate of 5%. <br>

*mde* / *power* / *n* are our key arguments of interest. We are interested in estimating the level of statistical power given a sample size of 20,000 and a minimum detectable effect of 4 units. We have the option of specifying whether we want to perform a one-tailed or two-tailed test by use of the the *alternative* argument. As a basis for comparsion, note that [statsmodels](https://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLSResults.pvalues.html) uses two-tailed t-statistics for reporting its p-values.

In [4]:
result_1 = osp.solve_power(is_ratio=False,
                           data=df,
                           endog='Y',
                           exog=[],
                           cluster='CLUSTER_ID',
                           ratio=0.5,
                           alpha=0.05,
                           mde=4,
                           #  power=0.8,
                           n=20000,
                           alternative="two-sided",
                           verbose=False)
result_1

No covariates were specified. Power analysis will be performed using only the intercept term.
NOTE: With no covariates, estimation will be akin to a t-test using pooled variance (i.e. a two-sample t-test). 
Are you sure this is the desired model specification?
Power analysis where the response variable is a non-ratio metric is as follows:
Statistical power was estimated as 0.785 using a minimum detectable effect (MDE) of 4. 
Estimate was based on 235,366 observations across 100,000 clusters of historical data. 
The model specification used 0 covariates, assumes 1 coefficient estimates, and 100,001 degrees of freedom (alpha=0.025).


0.784967043356559

In our first model specification, we leave the *exog* argument empty. This will estimate statistical power using the same assumptions as a t-test using the pooled variance assumption. *olspow* is estimating that our current specification will have a 78% probabiliy of successfully rejecting the null hypothesis (assumming that it is in fact, false). Now, let's see what adjusting for covariates X1 and X2 does.

In [5]:
result_2 = osp.solve_power(is_ratio=False,
                           data=df,
                           endog='Y',
                           exog=['X1_VALUE', 'X2_VALUE'],
                           cluster='CLUSTER_ID',
                           ratio=0.5,
                           alpha=0.05,
                           mde=4,
                           #  power=0.8,
                           n=20000,
                           alternative="two-sided",
                           verbose=False)
result_2

Power analysis where the response variable is a non-ratio metric is as follows:
Statistical power was estimated as 0.8648 using a minimum detectable effect (MDE) of 4. 
Estimate was based on 235,366 observations across 100,000 clusters of historical data. 
The model specification used 2 covariates, assumes 3 coefficient estimates, and 99,999 degrees of freedom (alpha=0.025).


0.8647728251598094

Using the same sample data as before, we've now added covariates X1 and X2 to the *exog* argument, adding them to the underlying regressions used to estimate the sample variance. If *verbose* argument is set to 'True,' users can see the outputs of the underlying regressions, and the fact that the R^2 of the expanded models exceed 0.9. This convariate adjustment has achieved significant variance reduction. Consequentially, the estimated statistical power has increased from to 78% over 86%.